<a href="https://colab.research.google.com/github/panthamramesh/Qualcomm-DL-Hackathon/blob/master/DL_Hackathon_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/Prashant-AV/Qualcomm-DL-Hackathon.git

Cloning into 'Qualcomm-DL-Hackathon'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 10 (delta 1), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (10/10), 30.68 MiB | 19.47 MiB/s, done.
Resolving deltas: 100% (1/1), done.


In [2]:
import os
import zipfile

In [3]:
import zipfile
import os
import shutil

def extract_files_only(zip_file_paths, destination_folder):
    # Create the destination folder if it doesn't exist
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)

    for zip_file_path in zip_file_paths:
        with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
            for member in zip_ref.namelist():
                # Check if the member is a file (not a directory)
                if not member.endswith('/'):
                    # Extract the file to a temporary location
                    source = zip_ref.open(member)
                    target_path = os.path.join(destination_folder, os.path.basename(member))
                    with open(target_path, "wb") as target:
                        shutil.copyfileobj(source, target)

# Example usage
zip_file_paths = ['/content/Qualcomm-DL-Hackathon/train/images part-1.zip', '/content/Qualcomm-DL-Hackathon/train/images part-2.zip']
destination_folder = '/content/Qualcomm-DL-Hackathon/Images'

extract_files_only(zip_file_paths, destination_folder)
print(f"Files from {zip_file_paths} have been extracted to {destination_folder}.")

Files from ['/content/Qualcomm-DL-Hackathon/train/images part-1.zip', '/content/Qualcomm-DL-Hackathon/train/images part-2.zip'] have been extracted to /content/Qualcomm-DL-Hackathon/Images.


In [4]:
def count_files_in_folder(folder_path):
    with os.scandir(folder_path) as entries:
        return sum(1 for entry in entries if entry.is_file())

# Example usage
folder_path = '/content/Qualcomm-DL-Hackathon/Images'
file_count = count_files_in_folder(folder_path)
print(f"Number of files in the folder: {file_count}")

Number of files in the folder: 2352


In [5]:
import os
import pandas as pd
from PIL import Image
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset,random_split
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn.metrics import confusion_matrix, classification_report

In [6]:
# Set device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [73]:
# Paths to the CSV files and image folder
train_csv_path = '/content/Qualcomm-DL-Hackathon/train/train.csv'
test_csv_path = '/content/Qualcomm-DL-Hackathon/test.csv'
image_folder_path = '/content/Qualcomm-DL-Hackathon/Images'

# Data transformations
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    #transforms.RandomHorizontalFlip(),
    #transforms.RandomRotation(30),
    transforms.ToTensor(),
    transforms.Normalize([0.406, 0.406, 0.406], [0.224, 0.224, 0.224])
])

# Load datasets
def load_data(csv_file, root_dir, transform):
    data_frame = pd.read_csv(csv_file)
    images = []
    labels = []
    for idx in range(len(data_frame)):
        img_name = os.path.join(root_dir, data_frame.iloc[idx, 0])
        image = Image.open(img_name)
        if transform:
            image = transform(image)
        images.append(image)
        if 'emergency_or_not' in data_frame.columns:
            labels.append(data_frame.iloc[idx, 1])
        else:
            labels.append(-1)
    return images, labels

train_images, train_labels = load_data(train_csv_path, image_folder_path, transform)
test_images, test_labels = load_data(test_csv_path, image_folder_path, transform)

train_data = list(zip(train_images, train_labels))
test_data = list(zip(test_images, test_labels))

# Split the training data into training and validation sets
train_size = int(0.8 * len(train_data))
val_size = len(train_data) - train_size
train_dataset, val_dataset = random_split(train_data, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

In [78]:
def create_model():
    model = nn.Sequential(
        #nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1),
        #nn.ReLU(),
        #nn.MaxPool2d(kernel_size=2, stride=2, padding=0),
        nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2, padding=0),
        nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2, padding=0),
        nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2, padding=0),
        nn.Flatten(),
        nn.Linear(128* 16 * 16, 512),
        nn.ReLU(),
        nn.Linear(512, 2)  # Assuming 2 classes: emergency and non-emergency
    )
    return model

model = create_model()

In [79]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [80]:
num_epochs = 10

def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        # Validate the model
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels in val_loader:
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        val_accuracy = 100 * correct / total
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, Val Loss: {val_loss/len(val_loader):.4f}, Val Accuracy: {val_accuracy:.2f}%')

train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs)

Epoch [1/10], Loss: 0.5932, Val Loss: 0.4299, Val Accuracy: 81.21%
Epoch [2/10], Loss: 0.4471, Val Loss: 0.4016, Val Accuracy: 80.30%
Epoch [3/10], Loss: 0.4024, Val Loss: 0.4125, Val Accuracy: 81.21%
Epoch [4/10], Loss: 0.3561, Val Loss: 0.3775, Val Accuracy: 80.30%
Epoch [5/10], Loss: 0.2889, Val Loss: 0.3522, Val Accuracy: 86.36%
Epoch [6/10], Loss: 0.2709, Val Loss: 0.3585, Val Accuracy: 85.15%
Epoch [7/10], Loss: 0.2393, Val Loss: 0.4020, Val Accuracy: 83.03%
Epoch [8/10], Loss: 0.1904, Val Loss: 0.3760, Val Accuracy: 85.76%
Epoch [9/10], Loss: 0.1625, Val Loss: 0.3879, Val Accuracy: 85.76%
Epoch [10/10], Loss: 0.1208, Val Loss: 0.3820, Val Accuracy: 86.97%


In [81]:
def compute_metrics(model, val_loader):
    model.eval()
    all_labels = []
    all_predictions = []

    with torch.no_grad():
        for images, labels in val_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())

    cm = confusion_matrix(all_labels, all_predictions)
    cr = classification_report(all_labels, all_predictions, target_names=['Non-Emergency', 'Emergency'])

    return cm, cr

# Compute confusion matrix and classification report using the validation set
cm, cr = compute_metrics(model, val_loader)
print("Confusion Matrix:")
print(cm)
print("\nClassification Report:")
print(cr)

Confusion Matrix:
[[169  15]
 [ 28 118]]

Classification Report:
               precision    recall  f1-score   support

Non-Emergency       0.86      0.92      0.89       184
    Emergency       0.89      0.81      0.85       146

     accuracy                           0.87       330
    macro avg       0.87      0.86      0.87       330
 weighted avg       0.87      0.87      0.87       330



In [82]:
def evaluate_model(model, test_loader):
    model.eval()
    predictions = []
    with torch.no_grad():
        for images, _ in test_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            predictions.extend(predicted.cpu().numpy())
    return predictions

predictions = evaluate_model(model, test_loader)

# Save predictions to CSV
test_df = pd.read_csv(test_csv_path)
test_df['emergency_or_not'] = predictions
test_df.to_csv('predictions_3.csv', index=False)

print("Predictions saved to predictions.csv")

Predictions saved to predictions.csv
